In [1]:
# Dependencies and Setup
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

### Import Opiate Mortaility Dataset

In [2]:
health04_S_file_path = "Resources/Health04/Health04_S.csv"
mortality_data = pd.read_csv(health04_S_file_path)

### Import Treatment Facility Dataset

In [3]:
file_path = "Resources/NSSATS/NSSATS_PUF_2020_CSV.csv"
facility_data = pd.read_csv(file_path, low_memory=False)

### Create DataFrames

In [4]:
mortality_df = pd.DataFrame(mortality_data)
mortality_variables = mortality_df.columns
mortality_df.head(3)

,G_STATEFP,STATEFP,st_abb,st_name,opPrscRt19,prMisuse19,odMortRt14,odMortRt15,odMortRt16,odMortRt17,odMortRt18,odMortRt19,odMortRtAv
0,G01,1,AL,Alabama,39.1,4.55,15.2,15.7,16.2,18.0,16.6,16.3,16.33
1,G02,2,AK,Alaska,85.8,3.99,16.8,16.0,16.8,20.2,14.6,17.8,17.03
2,G04,4,AZ,Arizona,80.9,3.80,18.2,19.0,20.3,22.2,23.8,26.8,21.72


In [5]:
facility_df = pd.DataFrame(facility_data)
facility_variables = list(facility_df.columns)
facility_df.head(3)

,CASEID,STATE,STFIPS,DETOX,TREATMT,SMISEDSUD,OWNERSHP,FEDOWN,HOSPITAL,LOCS,...,T_CLIHI_X,T_CLIML_D,T_CLIML_O,T_CLIML_X,T_CLIOP_D,T_CLIOP_O,T_CLIOP_X,T_CLIRC_D,T_CLIRC_O,T_CLIRC_X
0,1,AK,2,0,1,1,6,3.0,0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,4.0,NaN,NaN,NaN
1,2,AK,2,0,1,1,2,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,3,AK,2,1,1,0,2,NaN,0,NaN,...,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN


### Check State Abbreviations to compare and correct discrepancies

In [6]:
mortality_state = list(mortality_df['st_abb'].sort_values().unique())
len(mortality_state)

51

In [7]:
facility_state = list(facility_df['STATE'].sort_values().unique())
len(facility_state)

53

In [ ]:
mortality_state

In [ ]:
facility_state

In [14]:
discrepancy = []
for element in facility_state:
    if element not in mortality_state:
        discrepancy.append(element)
discrepancy

['PR', 'ZZ']

### PR & ZZ are in facility_state but not mortality_state
### -> remove from facility df

In [15]:
facility_df_pr = facility_df.loc[facility_df.STATE != 'PR']
facility_df_new = facility_df_pr.loc[facility_df_pr.STATE != 'ZZ']
facility_df_new

,CASEID,STATE,STFIPS,DETOX,TREATMT,SMISEDSUD,OWNERSHP,FEDOWN,HOSPITAL,LOCS,...,T_CLIHI_X,T_CLIML_D,T_CLIML_O,T_CLIML_X,T_CLIOP_D,T_CLIOP_O,T_CLIOP_X,T_CLIRC_D,T_CLIRC_O,T_CLIRC_X
0,1,AK,2,0,1,1,6,3.0,0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,4.0,NaN,NaN,NaN
1,2,AK,2,0,1,1,2,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,3,AK,2,1,1,0,2,NaN,0,NaN,...,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN
3,4,AK,2,0,1,1,2,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
4,5,AK,2,1,1,0,2,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16051,16052,WY,56,0,1,1,2,NaN,0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,4.0,NaN,NaN,NaN
16052,16053,WY,56,1,1,1,2,NaN,0,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,5.0,NaN,NaN,4.0
16053,16054,WY,56,0,1,1,2,NaN,0,NaN,...,NaN,1.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
16054,16055,WY,56,0,1,1,2,NaN,0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,3.0,NaN,NaN,NaN


In [16]:
facility_df_new.STATE.sort_values().unique() == mortality_state

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

### Prepare mortality data for merging with stats below:

In [17]:
mortality_df_alpha = mortality_df.sort_values(by='st_abb')
mortality_df_alpha.head(5)

,G_STATEFP,STATEFP,st_abb,st_name,opPrscRt19,prMisuse19,odMortRt14,odMortRt15,odMortRt16,odMortRt17,odMortRt18,odMortRt19,odMortRtAv
1,G02,2,AK,Alaska,85.8,3.99,16.8,16.0,16.8,20.2,14.6,17.8,17.03
0,G01,1,AL,Alabama,39.1,4.55,15.2,15.7,16.2,18.0,16.6,16.3,16.33
3,G05,5,AR,Arkansas,44.1,3.66,12.6,13.8,14.0,15.5,15.7,13.5,14.18
2,G04,4,AZ,Arizona,80.9,3.80,18.2,19.0,20.3,22.2,23.8,26.8,21.72
4,G06,6,CA,California,30.9,3.60,11.1,11.3,11.2,11.7,12.8,15.0,12.18


### Use Nested for Loop to Calculate Data on Facility Composition by State:

In [18]:
states = list(facility_df_new['STATE'].unique())

important_variables = ['DETOX', 'SMISEDSUD']

full_dict = {}
for state in states:
    subset = facility_df_new.loc[facility_df_new['STATE'] == state]
    stats = {}
    for variable in important_variables:
        y = subset[variable].value_counts()
        try:
            n_false = y[0]
        except:
            n_false = 0
        try:
            n_true = y[1]
        except:
            n_true = 0
        z = round(n_true/(n_false + n_true),2)
        stats[variable] = z
    full_dict[state] = stats
full_dict
stats_df = pd.DataFrame(full_dict)
stats_df_t = stats_df.T

stats_df_t.head(5)

,DETOX,SMISEDSUD
AK,0.11,0.69
AL,0.20,0.48
AR,0.15,0.72
AZ,0.23,0.67
CA,0.39,0.54


In [20]:
stats_df_t.to_csv("state_detox_smisedsud.csv")

### Create subset of only st_abb and mortality rate from mortality_df